In [163]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

In [164]:
fs = open("debug.sam","r")
debug_data=[]
for line in fs:
    debug_data.append(line.split('\t'))
fs.close()


# open original file

In [177]:
fs = open("SRR072893.sam","r")
data=[]
for line in fs:
    data.append(line.split('\t'))
fs.close()
data[1000]   

['SRR072893.19274188',
 '16',
 '2L',
 '12311',
 '60',
 '40M',
 '*',
 '0',
 '0',
 'CGGCATTCAGCTGTCTTTTTAATTCAGGTACTGATAAAAC',
 'CCCCCCCBCCCC@@CC@ACCCCACCCCCCCCCCCACCCCC',
 'AS:i:0',
 'XN:i:0',
 'XM:i:0',
 'XO:i:0',
 'XG:i:0',
 'NM:i:0',
 'MD:Z:40',
 'YT:Z:UU',
 'NH:i:1\n']

# 1. Count number of alignments

In [166]:
print("Test-alignment number:",len(debug_data))

Test-alignment number: 1000


In [167]:
print("Alignment number is:",len(data))

Alignment number is: 2000000


# 2. Count number of alignments that match perfectly to the genome


In [169]:
n=0
for i in range(len(data)):
    NM_list=data[i][-4].split(':')
    NM=NM_list[2]
    if NM == '0':
        n+=1
print('Test: Number of alignments that match perfectly:',n)  

Test: Number of alignments that match perfectly: 1578114


## Change another way to count the MN:i:0 as it may not always be the -4th item in the line. (The change was made after presentation session)

In [178]:
n=0
for i in data:
    if 'NM:i:0' in i:
        n+=1
print('Test: Number of alignments that match perfectly:',n)  

Test: Number of alignments that match perfectly: 1702765


### If NM is 0, it means there's no difference between reads and reference (mismatches plus inserted and deleted bases)

# 3. chromosomes

In [24]:
for i in range(10):
    print(data[i][2])

2L
2L
2L
2L
2L
2L
2L
2L
2L
2L


# 4. average MAPQ score 

In [28]:
MAPQ=[]
for i in range(len(debug_data)):
    MAPQ.append(float(debug_data[i][4]))
print("Test: average MAPQ score:",np.mean(MAPQ))    

Test: average MAPQ score: 59.289


In [30]:
MAPQ=[]
for i in range(len(data)):
    MAPQ.append(float(data[i][4]))
print("Average MAPQ score:",np.mean(MAPQ))    

Average MAPQ score: 59.6407825


# 5. Count number of reads that start their alignment on chromosome 2L between base 10000 and 20000 

In [179]:
n=0
for i in range(len(data)):
    start_pos=int(data[i][3])
#     length=int(len(data[i][9]))
    end_pos=start_pos
    if data[i][2] == '2L' and start_pos>=10000 and end_pos <=20000:
        n+=1
print("Number of reads starts between 2L 10000-20000:",n)
        

Number of reads starts between 2L 10000-20000: 7550


# Advanced 1

In [102]:
def get_bit(byteval,idx):
    return ((byteval&(1<<idx))!=0)
for i in range(len(data)):
    if get_bit(int(data[i][1]),4) ==True:
        n+=1
print("Number of reads map to the reverse strand:",n)

Number of reads map to the reverse strand: 990255


# Advanced 2

In [108]:
n=0
for i in range(len(data)):
    QUAL=data[i][10]
    score=[]
    for j in range(len(QUAL)):
        Q=ord(QUAL[j])-33
        score.append(Q)
    meanQ=np.mean(score)
    if meanQ>30:
        n+=1
print(n,"reads have an average quality score >30")
    


1386209 reads have an average quality score >30


# Advanced 3

In [171]:
n_1=0
n_2=0
n_3=0
n_4=0
n_5=0
for i in range(len(data)):
    CIGAR=data[i][5]
    number=re.findall("\d+", CIGAR)
    indels=['I','D','N','S','X','H']
    for indel in indels:
        cigar_list=[]
        for i in CIGAR: 
            if i.isalpha():
                cigar_list.append(i)     
            if indel in cigar_list:
                index=cigar_list.index(indel)
            else:
                index = -1
        if index >=0:
            indel_len=int(number[index])
            if indel_len == 1:
                n_1 +=1
            elif indel_len == 2:
                n_2 +=1
            elif indel_len == 3:
                n_3 +=1
            elif indel_len == 4:
                n_4 +=1
            else:
                n_5 +=1
                
   
print(n_1,"indels with length of 1")  
print(n_2,"indels with length of 2")  
print(n_3,"indels with length of 3")  
print(n_4,"indels with length of 4")  
print(n_5,"indels with length greater than 4")  
    
    

63944 indels with length of 1
30729 indels with length of 2
20842 indels with length of 3
15033 indels with length of 4
154027 indels with length greater than 4
